In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

# our code (mark it at autoreload at every cell execution - useful in developement mode)
%load_ext autoreload
%autoreload 1
%aimport runners_utils

In [ ]:
PATH_TO_DATA = '../../Scraping/DataSport/Data/Runners/'
# change the name to the global like website but need to see if the website still works.
RUNNERS_FILE = 'runners_2009.csv'
RUNS_FILE = 'runs_2009.csv'

In [ ]:
runners = pd.read_csv(PATH_TO_DATA + RUNNERS_FILE)
runs = pd.read_csv(PATH_TO_DATA + RUNS_FILE)

In [ ]:
runners.head()

In [ ]:
runs.head()

We remove useless columns, and all value showed that the runners have resigned.

In [ ]:
runners_utils.preprocess_runners(runners)

In [ ]:
runs = runners_utils.remove_outliers(runs)

For each races we compute the distance, we remove all other sports than running to remain coherent during the study.

In [ ]:
runs['distance (km)'] = runs.apply(runners_utils.compute_distance_from_category,axis=1)
runs = runs[~runs['distance (km)'].isnull()]

For each runners, we compute the gender (male/female).

In [ ]:
result = runs.apply(runners_utils.compute_sex, args=(runners,), axis=1)

In order to have enought races for each runners, we take only runners who have run more than 100 races throught 1999 to 2016.

In [ ]:
runs = runners_utils.select_runners_by_numbers_of_runs(runs, 100)

For each runners, we compute his age at the moment of the race.

In [ ]:
runs['age'] = runs.apply(runners_utils.compute_age, args=(runners,), axis=1)

In [ ]:
runs['time (s)'] = runs.apply(runners_utils.transform_string_to_second, axis=1)

In [ ]:
runs['eventDate'] = runs.apply(runners_utils.compute_date_event, axis=1)

In [ ]:
runners_utils.remove_useless_columns(runs)

After all operations of preprocessing, we obtain 10 runners who have finished more than 100 running races.

In [ ]:
len(runs['acode'].unique())

In [ ]:
runs.head(20)